In [1]:
def myKwargFunction(**kwargs):
    return kwargs

print(myKwargFunction(fuuga = 1, baari = 2))

params = {'foo' : 1, 'bar' : 'txt'}

myKwargFunction(**params)

{'fuuga': 1, 'baari': 2}


{'foo': 1, 'bar': 'txt'}

In [14]:
def curry(fun, **kwargsCurry):
    def ret(**kwargs):
        kwargsFun = kwargsCurry.copy()
        kwargsFun.update(kwargs)
        return fun(**kwargsFun)
    ret.defaultKwargs = kwargsCurry.copy()
    return ret

myCurriedFunction = curry(myKwargFunction, foo = 1)
myCurriedFunction(bar = 2), myCurriedFunction.defaultKwargs

({'foo': 1, 'bar': 2}, {'foo': 1})

In [19]:
import datetime

class myTime():
    def __init__(self, delta = 0):
        self.time = datetime.datetime.now()
    def __repr__(self):
        return(f"'{self.time:%Y-%m-%d %H:%M:%S.000}'")

def kwargsAsSQL(**kwargs):
    ret = []
    for key, valueRaw in kwargs.items():
#         if isinstance(valueRaw, str):
#             value = f"'valueRaw'"
#         else:
#             value = valueRaw
        value = valueRaw
        ret.append(f"{key} = {value!r} -- {type(valueRaw)}")
    return "    " + "\nand ".join(ret)

print(kwargsAsSQL(foo = 1
                  ,bar = 'string'
                  ,xyzzy = myTime()))

    foo = 1 -- <class 'int'>
and bar = 'string' -- <class 'str'>
and xyzzy = '2020-05-11 19:53:59.000' -- <class '__main__.myTime'>


In [20]:
import urllib
def parseQueryString(qs):
    return myKwargFunction(**urllib.parse.parse_qs(qs))
parseQueryString("foo=1&bar='ccc'&foo=2")

{'foo': ['1', '2'], 'bar': ["'ccc'"]}

In [23]:
def makeQueryString(**kwargs):
    return "&".join([f"{key}={value}" for key, value in kwargs.items()])

makeQueryString(**params)

'foo=1&bar=txt'

In [15]:
parseQueryString(makeQueryString(**params))

{'foo': ['1'], 'bar': ['txt']}

In [8]:
def makeQueryString(**kwargs):
    return urllib.parse.urlunparse(**kwargs)

makeQueryString(**params)

TypeError: urlunparse() got an unexpected keyword argument 'foo'

In [ ]:
def myCallFunctionWithQueryString(myFunction, myQueryString):
    myKwargs = urllib.parse.parse_qs(myQueryString)
    return myFunction(**myKwargs)

myCallFunctionWithQueryString(myFooBarFunction, "foo=1&bar='ccc'")

In [27]:
def foo(x = None, **kwargs):
    print(f"x = {x}, kwargs = {kwargs}")

myKwargs1 = {'y' : 2}
myKwargs2 = {'z' : 3}
myKwargs3 = {'z2' : 3}

foo(x = 1, **myKwargs1, **myKwargs2, **myKwargs3)

x = 1, kwargs = {'y': 2, 'z': 3, 'z2': 3}


In [15]:
def foo(a, b, c):
    return a * b + c

foo(1, 2, 3)
foo.__defaults__

In [22]:
def foo(a = 1, b = 2):
    return a * b

foo.__defaults__
foo.__kwdefaults__

In [11]:
print(foo.__closure__)

None


In [23]:
for attr in dir(foo):
    if attr == '__globals__':
        continue
    print(f"{attr} = {getattr(foo, attr)}")

__annotations__ = {}
__call__ = <method-wrapper '__call__' of function object at 0x00000171D3D72438>
__class__ = <class 'function'>
__closure__ = None
__code__ = <code object foo at 0x00000171D3D729C0, file "<ipython-input-22-c2c4064456b1>", line 1>
__defaults__ = (1, 2)
__delattr__ = <method-wrapper '__delattr__' of function object at 0x00000171D3D72438>
__dict__ = {}
__dir__ = <built-in method __dir__ of function object at 0x00000171D3D72438>
__doc__ = None
__eq__ = <method-wrapper '__eq__' of function object at 0x00000171D3D72438>
__format__ = <built-in method __format__ of function object at 0x00000171D3D72438>
__ge__ = <method-wrapper '__ge__' of function object at 0x00000171D3D72438>
__get__ = <method-wrapper '__get__' of function object at 0x00000171D3D72438>
__getattribute__ = <method-wrapper '__getattribute__' of function object at 0x00000171D3D72438>
__gt__ = <method-wrapper '__gt__' of function object at 0x00000171D3D72438>
__hash__ = <method-wrapper '__hash__' of function o

In [19]:
foo.__reduce__()

TypeError: can't pickle function objects

In [42]:
def as_args_function(function, keywords):
    def ret(*args):
        kwargs = {}
        for arg, kw in zip(args, keywords):
            kwargs[kw] = arg
        return function(**kwargs)
    return ret
    
def foo(**kwargs):
    return kwargs['a'] ** kwargs['b']
    
assert foo(a = 2, b = 3) == 8
assert foo(a = 3, b = 2) == 9

bar_1 = as_args_function(foo, ['a', 'b'])
assert bar_1(2, 3) == 8

bar_2 = as_args_function(foo, ['b', 'a'])
assert bar_2(2, 3) == 9


In [77]:
import re

def gen_args_format(txt, variable_pattern = r':([a-z_]+)'):
    """
    Return a function that formats txt with given arguments. Arguments 
    """
    matches = re.findall(variable_pattern, txt)
    variable_names = list(dict.fromkeys(matches))
    format_pattern = r'{\1!r}'
    txt_format = re.sub(variable_pattern, format_pattern, txt)
    return as_args_function(txt_format.format, variable_names)



gen_args_format('If :a > :b then :b < :a')(2, 1)

'If 2 > 1 then 1 < 2'

In [72]:
testKwargs = {'c' : 1, 'b' : 2, 'd_e' : 44}
testTxtFormat.format(c = 1, b = 2, d_e = 44)
testTxtFormat.format(**testKwargs)

'\na 1 foo 2 d 1 2 44\n'

In [74]:
myFunction(33, 4, 5)

'\na 33 foo 4 d 33 4 5\n'